In [1]:
import os

In [2]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.sequential import SequentialChain
from langchain.callbacks import get_openai_callback
import json
import pandas as pd
import traceback
from dotenv import load_dotenv   
import PyPDF2

In [3]:
load_dotenv()

True

In [4]:
open_key = os.getenv('openai_key')

In [5]:
llm = ChatOpenAI(openai_api_key = open_key, model_name='gpt-3.5-turbo',temperature =0.7)

f:\ML\NLP\MCQ-Generator\venv\Lib\site-packages\langchain_core-0.1.52-py3.11.egg\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
llm

In [6]:
template = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}
"""

In [7]:
prompt_template = PromptTemplate(
    input_variables=['text','number','subject','tone','RESPONSE_JSON'],
    template=template
)

In [8]:
with open('F:/ML/NLP/MCQ-Generator/Response.json','r') as f:
    RESPONSE_JSON = json.load(f)

In [9]:
RESPONSE_JSON

{'1': {'no': '1',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'no': '2',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'no': '3',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}

In [10]:
quiz_chain = LLMChain(llm =llm, prompt=prompt_template, output_key = 'quiz',verbose=True)

f:\ML\NLP\MCQ-Generator\venv\Lib\site-packages\langchain_core-0.1.52-py3.11.egg\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [11]:
template2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [12]:
evalution_prompt_template = PromptTemplate(
    input_variables=['subject','quiz'],
    template=template2
)

In [13]:
review_chain = LLMChain(llm =llm, prompt=evalution_prompt_template, output_key = 'review',verbose=True)

In [14]:
generate_evaluate_chain = SequentialChain(chains=[quiz_chain,review_chain],input_variables=['text','number','subject','tone','RESPONSE_JSON'],output_variables=['quiz','review'])

In [15]:
# paragraph for MCQ generation
with open('F:/ML/NLP/MCQ-Generator/data.txt') as f:
    text = f.read()

In [16]:
text

'Cricket is a bat-and-ball game that is played between two teams of eleven players on a field at the centre of which is a 22-yard (20-metre) pitch with a wicket at each end, each comprising two bails balanced on three stumps. Two players from the batting team (the striker and nonstriker) stand in front of either wicket, with one player from the fielding team (the bowler) bowling the ball towards the striker\'s wicket from the opposite end of the pitch. The striker\'s goal is to hit the bowled ball and then switch places with the nonstriker, with the batting team scoring one run for each exchange. Runs are also scored when the ball reaches or crosses the boundary of the field or when the ball is bowled illegally.\n\nThe fielding team tries to prevent runs from being scored by dismissing batters (so they are "out"). Means of dismissal include being bowled, when the ball hits the striker\'s wicket and dislodges the bails, and by the fielding side either catching the ball after it is hit b

In [17]:
number = 5
subject = 'Sport'
tone = 'Simple'

In [40]:
import json
RESPONSE_JSON = json.dumps(RESPONSE_JSON)

In [41]:
RESPONSE_JSON

'{"1": {"no": "1", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"no": "2", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"no": "3", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [42]:

with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
        'text':text,
        'number':number,
        'subject':subject,
        'tone':tone,
        'RESPONSE_JSON':RESPONSE_JSON
        }
    )



> Entering new LLMChain chain...
Prompt after formatting:

Text:Cricket is a bat-and-ball game that is played between two teams of eleven players on a field at the centre of which is a 22-yard (20-metre) pitch with a wicket at each end, each comprising two bails balanced on three stumps. Two players from the batting team (the striker and nonstriker) stand in front of either wicket, with one player from the fielding team (the bowler) bowling the ball towards the striker's wicket from the opposite end of the pitch. The striker's goal is to hit the bowled ball and then switch places with the nonstriker, with the batting team scoring one run for each exchange. Runs are also scored when the ball reaches or crosses the boundary of the field or when the ball is bowled illegally.

The fielding team tries to prevent runs from being scored by dismissing batters (so they are "out"). Means of dismissal include being bowled, when the ball hits the striker's wicket and dislodges the bails, and by 

In [43]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost}")

Total Tokens: 1797
Prompt Tokens: 1339
Completion Tokens: 458
Total Cost (USD): $0.0029245


In [44]:
quiz = response.get('quiz')
quiz

'\n{\n  "1": {\n    "no": "1",\n    "mcq": "What is the main objective of the fielding team in cricket?",\n    "options": {\n      "a": "To score runs",\n      "b": "To dismiss batters",\n      "c": "To hit boundaries",\n      "d": "To bowl the ball"\n    },\n    "correct": "b"\n  },\n  "2": {\n    "no": "2",\n    "mcq": "What is the primary equipment used by batters to protect themselves from injury?",\n    "options": {\n      "a": "Helmet",\n      "b": "Gloves",\n      "c": "Pads",\n      "d": "All of the above"\n    },\n    "correct": "d"\n  },\n  "3": {\n    "no": "3",\n    "mcq": "Which organization is responsible for maintaining the Laws of Cricket?",\n    "options": {\n      "a": "International Cricket Council (ICC)",\n      "b": "Marylebone Cricket Club (MCC)",\n      "c": "Federation of International Cricket Associations (FICA)",\n      "d": "World Cricket Board (WCB)"\n    },\n    "correct": "b"\n  },\n  "4": {\n    "no": "4",\n    "mcq": "In which region is cricket primarily

In [45]:
type(quiz)

str

In [47]:
quiz = json.loads(quiz)
quiz

{'1': {'no': '1',
  'mcq': 'What is the main objective of the fielding team in cricket?',
  'options': {'a': 'To score runs',
   'b': 'To dismiss batters',
   'c': 'To hit boundaries',
   'd': 'To bowl the ball'},
  'correct': 'b'},
 '2': {'no': '2',
  'mcq': 'What is the primary equipment used by batters to protect themselves from injury?',
  'options': {'a': 'Helmet',
   'b': 'Gloves',
   'c': 'Pads',
   'd': 'All of the above'},
  'correct': 'd'},
 '3': {'no': '3',
  'mcq': 'Which organization is responsible for maintaining the Laws of Cricket?',
  'options': {'a': 'International Cricket Council (ICC)',
   'b': 'Marylebone Cricket Club (MCC)',
   'c': 'Federation of International Cricket Associations (FICA)',
   'd': 'World Cricket Board (WCB)'},
  'correct': 'b'},
 '4': {'no': '4',
  'mcq': 'In which region is cricket primarily followed?',
  'options': {'a': 'North America',
   'b': 'Europe',
   'c': 'South Asia',
   'd': 'South America'},
  'correct': 'c'},
 '5': {'no': '5',
  'mc

In [48]:
type(quiz)

dict

In [60]:
type(quiz.items())

dict_items

In [63]:

for key,value in quiz.items():
    print(value)

{'no': '1', 'mcq': 'What is the main objective of the fielding team in cricket?', 'options': {'a': 'To score runs', 'b': 'To dismiss batters', 'c': 'To hit boundaries', 'd': 'To bowl the ball'}, 'correct': 'b'}
{'no': '2', 'mcq': 'What is the primary equipment used by batters to protect themselves from injury?', 'options': {'a': 'Helmet', 'b': 'Gloves', 'c': 'Pads', 'd': 'All of the above'}, 'correct': 'd'}
{'no': '3', 'mcq': 'Which organization is responsible for maintaining the Laws of Cricket?', 'options': {'a': 'International Cricket Council (ICC)', 'b': 'Marylebone Cricket Club (MCC)', 'c': 'Federation of International Cricket Associations (FICA)', 'd': 'World Cricket Board (WCB)'}, 'correct': 'b'}
{'no': '4', 'mcq': 'In which region is cricket primarily followed?', 'options': {'a': 'North America', 'b': 'Europe', 'c': 'South Asia', 'd': 'South America'}, 'correct': 'c'}
{'no': '5', 'mcq': "What is the name of the game's governing body?", 'options': {'a': 'FIFA', 'b': 'ICC', 'c': 

In [68]:
quiz_table = []

for key,value in quiz.items():
    QNO = value['no']
    mcq = value['mcq']
    options = " | ".join(
        #list comprehension
        [
            f"{option}:{option_value}"
            for option , option_value in value['options'].items()
        ]
    )

    correct = value['correct']
    quiz_table.append({"Q.NO.":QNO,"MCQ":mcq,"Choices":options,"Correct":correct})

In [69]:
quiz_table

[{'Q.NO.': '1',
  'MCQ': 'What is the main objective of the fielding team in cricket?',
  'Choices': 'a:To score runs | b:To dismiss batters | c:To hit boundaries | d:To bowl the ball',
  'Correct': 'b'},
 {'Q.NO.': '2',
  'MCQ': 'What is the primary equipment used by batters to protect themselves from injury?',
  'Choices': 'a:Helmet | b:Gloves | c:Pads | d:All of the above',
  'Correct': 'd'},
 {'Q.NO.': '3',
  'MCQ': 'Which organization is responsible for maintaining the Laws of Cricket?',
  'Choices': 'a:International Cricket Council (ICC) | b:Marylebone Cricket Club (MCC) | c:Federation of International Cricket Associations (FICA) | d:World Cricket Board (WCB)',
  'Correct': 'b'},
 {'Q.NO.': '4',
  'MCQ': 'In which region is cricket primarily followed?',
  'Choices': 'a:North America | b:Europe | c:South Asia | d:South America',
  'Correct': 'c'},
 {'Q.NO.': '5',
  'MCQ': "What is the name of the game's governing body?",
  'Choices': 'a:FIFA | b:ICC | c:NBA | d:NFL',
  'Correct': 

In [70]:
type(quiz_table)

list

In [72]:
df = pd.DataFrame(quiz_table)
df

,Q.NO.,MCQ,Choices,Correct
0,1,What is the main objective of the fielding tea...,a:To score runs | b:To dismiss batters | c:To ...,b
1,2,What is the primary equipment used by batters ...,a:Helmet | b:Gloves | c:Pads | d:All of the above,d
2,3,Which organization is responsible for maintain...,a:International Cricket Council (ICC) | b:Mary...,b
3,4,In which region is cricket primarily followed?,a:North America | b:Europe | c:South Asia | d:...,c
4,5,What is the name of the game's governing body?,a:FIFA | b:ICC | c:NBA | d:NFL,b


In [73]:
df.to_csv('quiz_cricket.csv',index = False)

In [1]:

log_file = "{datetime.now().strtime('%m_%d_%Y_%H_%M_%S')}.log"


In [2]:
log_file

"{datetime.now().strtime('%m_%d_%Y_%H_%M_%S')}.log"

In [13]:
from datetime import datetime
datetime.now().strftime("%d/%m/%Y %H:%M:%S")

'11/05/2024 10:53:06'

In [14]:
datetimeFormat = '%d/%m/%Y %H:%M:%S'
log_file = "{datetime.now().strftime({datetimeFormat})}.log"

In [17]:
datetime.now().strftime(datetimeFormat)

'11/05/2024 10:59:39'